# SQLite Database Creation
To load and manipulate the large `DATASET1_CellTrialTable.csv` (~117 GB), we are going to load it into an SQL database and create an index for the `Experiment` column.\
Be warned, this script will take 5 hours to run.

## Imports

In [ ]:
import sqlite3
import pandas as pd
from tqdm import tqdm

## Create Database

In [ ]:
def create_sqlite_db_from_csv(csv_file, db_file, table_name):
    """
    Create an SQLite database from a CSV file, preserving all columns and column types.
    
    Args:
    - csv_file (str): Path to the CSV file.
    - db_file (str): Path to the SQLite database file.
    - table_name (str): Name of the table to create in the SQLite database.
    """
    # Step 1: Read a sample of the CSV to infer the schema
    sample_size = 1000  # Number of rows to read for inferring the schema
    sample_df = pd.read_csv(csv_file, nrows=sample_size)
    
    # Step 2: Create a dictionary mapping pandas dtypes to SQLite types
    dtype_mapping = {
        'object': 'TEXT',
        'int64': 'INTEGER',
        'float64': 'REAL',
        'datetime64[ns]': 'TEXT',  # Dates can be stored as TEXT in SQLite
        'bool': 'INTEGER'  # SQLite does not have a separate BOOLEAN type
    }
    
    # Step 3: Infer the SQLite column types based on the sample
    column_types = {}
    for column, dtype in sample_df.dtypes.items():
        column_types[column] = dtype_mapping.get(str(dtype), 'TEXT')  # Default to TEXT if dtype is not in mapping
    
    # Step 4: Create the SQLite table based on the inferred schema
    columns_with_types = ', '.join([f'"{col}" {dtype}' for col, dtype in column_types.items()])
    create_table_sql = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types});'
    
    # Step 5: Connect to SQLite database and create the table
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute(create_table_sql)
    conn.commit()
    
    print(f"Table '{table_name}' created successfully with columns: {list(column_types.keys())}")
    
    # Step 6: Read and insert the CSV data into SQLite in chunks
    chunk_size = 100000  # Number of rows per chunk (adjust based on your memory constraints)
    for chunk in tqdm(pd.read_csv(csv_file, chunksize=chunk_size)):
        chunk.to_sql(table_name, conn, if_exists='append', index=False)

    # Step 7: Close the connection
    conn.close()
    print(f"Data from '{csv_file}' has been successfully imported into '{db_file}' as table '{table_name}'.")

In [ ]:
csv_file = '/home/jovyan/work/dataset_corbo/DATASET1_CellTrialTable.csv'
db_file = '/home/jovyan/work/dataset_corbo/DATASET1_CellTrialTable.db'
table_name = 'CellTrialTable'  # Name of the table in the SQLite database

create_sqlite_db_from_csv(csv_file, db_file, table_name)

## Create `Index` for `Experiment` column

In [ ]:
conn = sqlite3.connect('/home/jovyan/work/dataset_corbo/DATASET1_CellTrialTable.db')
cursor = conn.cursor()
cursor.execute("CREATE INDEX IF NOT EXISTS idx_experiment ON CellTrialTable (Experiment);")
conn.commit()
conn.close()